In [1]:
import numpy as np, scipy.integrate, scipy.optimize, sys



In [2]:
 def __init__(self, beta, gamma, S, I, R):
        self.beta = beta
        self.gamma = gamma
        self.t = 0.
        self.S = S
        self.I = I
        self.R = R
        self.N = S+I+R
        self.trajectory = np.array([[self.S, self.I, self.R]],dtype=float)/self.N
        self.times = None

def reset(self, S, I, R, t=0.):
        self.t = t
        self.S = S
        self.I = I
        self.R = R
        self.trajectory = np.array([[self.S, self.I, self.R]],dtype=float)/self.N
        
def get_total_infected(self):
        """Return the total number of hosts either currently or previously infected"""
        return self.I + self.R

In [3]:
def step(self):
    transition = None
    inf_rate = (self.beta * self.S * self.I)/self.N
    rec_rate = self.gamma * self.I
    total_rate = inf_rate + rec_rate
    if total_rate == 0.:
        return transition, self.t
    ranno = np.random.random()
    if ranno < inf_rate/total_rate:
        self.S -= 1
        self.I += 1
        transition = 1
    else:
        self.I -= 1
        self.R += 1
        transition = 2
        dt = np.random.exponential(1./total_rate, 1)[0]
        self.t += dt
        return transition, self.t
def run(self, T=None, make_traj=True):
        """Run the Gillespie algorithm for stochastic simulation from time 0 to at least time T, starting with the initial
        values stored in the S,I,R state variables; story the result in self.trajectory if make_traj argument is
        set to True"""

        if T is None:
            T = sys.maxsize
        self.times = [0.]
        t0 = self.t
        transition = 1
        while self.t < t0 + T:
            transition, t = self.step()
            if not transition:
                return self.t
            if make_traj: self.trajectory = np.concatenate((self.trajectory, [[self.S,self.I,self.R]]), axis=0)
            self.times.append(self.t)
        return self.t
    
def StochasticOutbreakSizeDistribution(N, beta, gamma, Nruns):
    """For a given population size N and model parameters beta and gamma, execute Nruns dynamical runs of the
    StochasticSIRsystem to calculate the overall size of each outbreak (total number of hosts infected) after it has
    finally died out; store the outbreak sizes in an array, which is returned for further characterization."""

    S,I,R = N-1, 1, 0
    ssir = StochasticSIRsystem(beta, gamma, S, I, R)
    sizes = []
    durations = []
    for n in range(Nruns):
        ssir.reset(S,I,R)
        t_end = ssir.run(make_traj=False)
        outbreak_size = ssir.get_total_infected()
        sizes.append(outbreak_size)
        durations.append(t_end)
    return np.array(sizes), np.array(durations)


In [4]:
def demo():
    import pylab
    N = 1000
    print("SIR demo")
    print("Deterministic SIR dynamics")
    pylab.figure(1)
    pylab.clf()
    dsir = DeterministicSIRsystem(1.5, 1.0, N-1, 1, 0)
    dsir.run(30, 0.1)
    pylab.plot(dsir.times, dsir.trajectory[:,0], 'b-', label='S')
    pylab.plot(dsir.times, dsir.trajectory[:,1], 'r-', label='I')
    pylab.plot(dsir.times, dsir.trajectory[:,2], 'g-', label='R')
    pylab.legend(loc='upper right')
    if not yesno(): return
    print("Deterministic outbreak size")
    R0_range = np.arange(0.,5.,0.1)
    simulated_sizes = SimulateDeterministicOutbreakSize(N=N, R0_range=R0_range)
    theoretical_sizes = CalculateDeterministicOutbreakSize(N=N, R0_range=R0_range)
    pylab.figure(2)
    pylab.clf()
    pylab.plot(R0_range, N*theoretical_sizes[:,1], 'b-', label='theory')
    pylab.plot(R0_range, simulated_sizes[:,1], 'bo', label='simulations')
    pylab.xlabel('R0')
    pylab.ylabel('total outbreak size')
    if not yesno(): return
    print("Stochastic SIR dynamics")
    pylab.figure(3)
    pylab.clf()
    pylab.plot(dsir.times, N*dsir.trajectory[:,1], 'r-', linewidth=2)
    for n in range(20):
        ssir = StochasticSIRsystem(1.5, 1.0, N-1, 1, 0)
        tfinal = ssir.run(20)
        pylab.plot(ssir.times, ssir.trajectory[:,1], 'b-')
    if not yesno(): return
    